# Clean the data set step by step

In [166]:
import numpy as np
import pandas as pd
import re
from dateutil.parser import parse
import warnings


In [167]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [168]:
df = pd.read_csv("../data/attacks.csv", encoding='latin1')

In [169]:
# rename columns
df.rename(columns={'Sex ':'Sex', 'Species ':'Species'}, inplace=True)

In [170]:
# no data from 6302 on
df = df.drop(df.index[6302:])

In [171]:
# drop duplicated
df.drop([4688, 5709, 6295], inplace=True)

In [172]:
# unique values for the identifier column
df['Case Number'] = [int(e) for e in range(1, len(df)+1)]

In [173]:
# case number is date till row 6300
df.loc[0:6300, 'Date'] = df.loc[0:6300, 'Case Number.1']

# kepp only the wanted characters
df['Date'] = df['Date'].str.extract('^(\d{4}\.\d{2}\.\d{2})', expand=False)

# rest is unknown
df.loc[6177:, 'Date'] = 'unknown'


In [174]:
# years have '.0' at the end
df['Year'] = df['Year'].astype(str).replace(to_replace=r'\.0', value='', regex=True)

# Replace years that are not 4 digits with NaN
df['Year'] = df['Year'].apply(lambda x: x if len(str(x)) == 4 else np.nan)

# Convert to numeric, making non-numeric values NaN
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Cast into integer
df['Year'] = df['Year'].astype('Int64')

#importany: years go from 1543 to 2018
df['Year'].min()

1543

In [175]:
# null is 'unknown'
df['Type'].fillna('unknown', inplace=True)
# invalid is unknown
df['Type'] = df['Type'].replace(to_replace='Invalid', value='unknown', regex=True)

# standarize boating
df['Type'] = df['Type'].replace(to_replace='Boat', value='Boating')
df['Type'] = df['Type'].replace(to_replace='Boatomg', value='Boating')

In [176]:
#too many unique values
df['Species'].fillna('unknown', inplace=True)
df['Investigator or Source'].fillna('unknown', inplace=True)

In [177]:
# clean countries
df['Country'] = df['Country'].str.capitalize()

df['Country'].replace({'St helena, british overseas territory': 'British overseas territory',
  'Diego garcia': 'unknown', 
  'Turks & caicos': 'Turkish islands and caicos',
  'United arab emirates (uae)': 'United arab emirates',
  'British virgin islands': 'British overseas territory',
  'Grand cayman': 'Caiman Islands',
  'Mexico ': 'Mexico',
  ' tonga': 'Tonga',
  'Egypt / israel': 'Egypt',
  'British isles': 'British overseas territory',
  'Federated states of micronesia': 'Micronesia',
  'Admiralty islands': 'Papua New Guinea',
  'Red sea / indian ocean': 'Read sea',
  'Andaman / nicobar islandas': 'Andaman and nicobar islands',
  'Sudan?': 'Sudan',
  'Iran / iraq': 'Iran',
  ' philippines': 'Philippines',
  'Solomon islands / vanuatu': 'Solomon islands',
  'Southwest pacific ocean': 'South pacific ocean',
  'Mid-pacifc ocean': 'Mid pacifc ocean',
  'Italy / croatia': 'Italy',
  'San domingo': 'Dominican republic',
  'Crete': 'Greece',
  'Egypt ': 'Egypt',
  'British new guinea': 'Papua New Guinea',
  'Ocean': 'unknown',
  'Indian ocean?': 'Indian ocean',
  'Andaman islands': 'Andaman and nicobar islands',
  'Equatorial guinea / cameroon': 'Equatorial guinea',
  'Africa': 'unknown',                    
  'Coast of africa': 'unknown',
  'Between portugal & india': 'unknown',
  'Red sea?': 'Read sea',
  'Asia?': 'unknown',
  'Ceylon (sri lanka)': 'Ceylon'
  }, inplace=True)

df['Country'].fillna('unknown', inplace=True)

In [178]:
#too many unique values
df['Area'] = df['Area'].str.capitalize()

df['Area'].fillna('unknown', inplace=True)

In [179]:
df.query("Country == 'unknown' & Area != 'unknown'")

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
2956,2957,1983.00.00,1983,Unprovoked,unknown,English channel,NaN,Swimming,Padma Shri Taranath Narayan Shenoy,M,NaN,Left leg bitten,N,NaN,unknown,"Times of India, 2/5/2012",1983.00.00.d-Shenoy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1983.00.00.d,1983.00.00.d,3347.0,NaN,NaN
3387,3388,1970.08.02,1970,unknown,unknown,Caribbean sea,Between St. Kitts & Nevis,Sea Disaster Sinking of ferryboat Christina,NaN,NaN,NaN,"Sharks scavenged on bodies, but no record of t...",NaN,Afternoon,Shark involvement prior to death was not confi...,"Rome News Tribune, 8/3/1970",1970.08.02-Christina-ferryboat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.08.02,1970.08.02,2916.0,NaN,NaN
4018,4019,1960.01.26,1960,Sea Disaster,unknown,"Between timor & darwin, australia",NaN,Portuguese Airliner with 9 people aboard went ...,NaN,NaN,NaN,"As searchers approached wreckage, sharks circl...",N,NaN,unknown,"V.M. Coppleson (1962), p.260",1960.01.26-Portuguese airliner.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1960.01.26,1960.01.26,2285.0,NaN,NaN
4231,4232,1956.09.13,1956,Unprovoked,unknown,Near the andaman & nicobar islands,NaN,Climbing back on ship,male,M,NaN,FATAL,Y,P.M.,Blue shark,M. Hosina,1956.09.13-TunaBoat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1956.09.13,1956.09.13,2072.0,NaN,NaN
4266,4267,1956.00.00,1956,Sea Disaster,unknown,Between comores & madagascar,Geyser Bank,Shipwreck,"Captain Eric Hunt, the cook & a French passenger",M,NaN,FATAL,Y,NaN,unknown,dinofish.com,1956.00.00.g-Capt-Hunt.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1956.00.00.g,1956.00.00.g,2037.0,NaN,NaN
4498,4499,1949.12.00,1949,Sea Disaster,unknown,Caribbean sea,Between Cuba & Costa Rica,"Sea Disaster, sinking of the motorship Wingate","Albert Battles, James Dean & 4 crew",M,NaN,Fatal or drowning or scavenging,Y,NaN,Shark involvement not confirmed,"Canberra Times, 1/6/1950",1949.12.00.b-Wingate.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1949.12.00.b,1949.12.00.b,1805.0,NaN,NaN
4700,4700,1942.11.00,1942,Sea Disaster,unknown,Off south american coast,NaN,Dutch merchant ship Zaandam torpedoed by the ...,NaN,M,NaN,FATAL,Y,NaN,unknown,"M. Murphy; V.M. Coppleson (1962), pp.207-208",1942.11.00.a-Izzi.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1942.11.00.a,1942.11.00.a,1603.0,NaN,NaN
4712,4712,1942.06.00,1942,Unprovoked,unknown,300 miles east of st. thomas (virgin islands),NaN,On life raft tethered to lifeboat. A seaman pu...,male,M,NaN,Forearm lacerated,N,NaN,unknown,"V.M. Coppleson (1962), p.258",1942.06.00-on-life-raft.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1942.06.00,1942.06.00,1591.0,NaN,NaN
5020,5020,1931.04.27,1931,Unprovoked,unknown,French southern territories,Île Saint-Paul,"Fishing, boat capsized",Quillezic,M,NaN,FATAL,Y,NaN,unknown,"Los Angeles Times, 4/27/1931",1931.04.27.R-Quillezic.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1931.04.27.R,1931.04.27.R,1283.0,NaN,NaN
5612,5612,1897.03.15,1897,Unprovoked,unknown,Mediterranean sea,NaN,Swimming,male,M,NaN,FATAL,Y,NaN,unknown,"Daily Northwestern, 5/15/1897",1897.03.15.b.R-Mediterranean.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1897.03.15.b.R,1897.03.15.b.R,691.0,NaN,NaN


In [180]:
# countreis that can be deduced from the area as shown above
nuevos_valores = {2956: 'English channel',
  3387: 'Caribbean sea',
  4018: 'Australia',
  4231: 'Andaman and nicobar islands',
  4266: 'Madagascar',
  4498: 'Caribbean sea',
  4712: 'Caribbean sea',
  5020: 'Indian ocean',
  5612: 'Mediterranean sea',
  5742: 'Mexico',
  5748: 'Northern pacific ocean',
  6059: 'Caribbean sea',
  6137: 'Middle atlantic ocean',
  6175: 'Ionian sea',
  6177: 'Ionian sea',              
  }

df['Country'].update(pd.Series(nuevos_valores))

In [181]:
#too many unique values
df['Location'] = df['Location'].str.capitalize()

df['Location'].fillna('unknown', inplace=True)

In [182]:
# too many unique values
df['Activity'].fillna('Activity', inplace=True)

In [183]:
df.Sex.fillna('unknown', inplace=True)

df.Sex.replace({'M ': 'M',
  'lli': 'unknown', 
  'N': 'unknown',
  '.': 'unknown'
  }, inplace=True)

In [184]:
df['Name'].fillna('unknown', inplace=True)

df['Name'].replace({'Unknown': 'unknown'} , inplace=True)

In [185]:
df.Sex.fillna('unknown', inplace=True)

df.Sex.replace({'M ': 'M',
  'lli': 'unknown', 
  'N': 'unknown',
  '.': 'unknown'
  }, inplace=True)

In [186]:
df.query("Sex == 'unknown' & Name != 'unknown'")

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
59,60,2017.11.25,2017,Sea Disaster,Libya,unknown,gars garabulli,2 boats capsized,31 migrants,unknown,NaN,FATAL,Y,NaN,Some drowned but other may have been killed by...,"TG Com 24, 11/25/2017",2017.11.25.R-Libya.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.11.25.R,2017.11.25.R,6244.0,NaN,NaN
179,180,2017.02.01,2017,Boating,Usa,South carolina,16 miles off hilton head,Tagging sharks,Chip Michelove & crew,unknown,NaN,"Shark bit boat, no injury to occupants",N,NaN,"White shark, female, 14'","YouTube, 2/2/2017",2017.02.01.b-Boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.02.01.b,2017.02.01.b,6124.0,NaN,NaN
218,219,2016.09.15,2016,Boating,Australia,Western australia,Bunbury,Fishing,Occupant: Ben Stratton,unknown,NaN,Shark rammed boat. No injury to occupant,N,NaN,unknown,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,6085.0,NaN,NaN
243,244,2016.07.17,2016,Boating,Usa,Alabama,8 miles off mobile,Fishing in Alabama Deep Fishing Rodeo,Occupant: Ben Raines,unknown,NaN,"No injury, shark bit trolling motor",N,NaN,"Tiger shark, 10'","Al.com, 7/19/2016",2016.07.17-Gulf.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.17,2016.07.17,6060.0,NaN,NaN
326,327,2015.12.13,2015,Boating,Australia,New south wales,Lake macquarie,Fishing,6 m boat: occupants Stephen & Andrew Crust,unknown,NaN,"No injury, shark rammed boat & bit motor",N,10h30,"White shark, 3.5 m","Courier Mail, 12/15/2015",2015.12.13-Crust-Boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.12.13,2015.12.13,5977.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6167,6166,1617.00.00,1617,Unprovoked,India,West bengal,Ganges delta,Activity,Indian people,unknown,NaN,NaN,UNKNOWN,NaN,unknown,"H. Edwards, p.31, citing Samuel Purchas",1617.00.00-Purchas.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1617.00.00.R,1617.00.00.R,136.0,NaN,NaN
6202,6201,unknown,<NA>,Unprovoked,Djibouti,Gulf of tadjoura,unknown,A dhow capsized,Passenger & crew,unknown,NaN,FATAL,Y,NaN,unknown,A. C. Doyle,ND-0119-Gulf-of-Tadjoura.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND-0119,ND-0119,101.0,NaN,NaN
6212,6211,unknown,<NA>,Boating,Mozambique,Inhambane province,Off inhambane,Fishing,"4.8-metre skiboat, Occupants: Rod Salm & 4 fri...",unknown,NaN,"No injury to occupants, shark bumped boat",N,NaN,Whale shark,South African Shark Attack File,ND-0107-Inhambane.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND-0107,ND-0107,91.0,NaN,NaN
6259,6258,unknown,<NA>,Sea Disaster,unknown,unknown,unknown,A group of survivors on a raft for 17-days,C.,unknown,NaN,"FATAL, shark leapt into raft and bit the man w...",Y,Late afternoon,1.2 m [4'] shark,"G.A. Llano in Airmen Against the Sea, p.69",ND-0044-C.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0044,ND.0044,44.0,NaN,NaN


In [187]:
# gender can be deduced by the name
nuevos_valores_area = {218: 'M',
  243: 'M',
  463: 'M', 
  465: 'M',
  747: 'M',
  820: 'M', 
  839: 'M',
  1177: 'M',
  1366: 'M', 
  1525: 'F',
  1624: 'M',
  2225: 'M', 
  2246: 'M',
  2273: 'M',
  2279: 'M', 
  2295: 'M',
  2317: 'M',
  2452: 'F', 
  2472: 'M',
  2478: 'M',
  2502: 'M', 
  2505: 'M',
  2799: 'M',
  2865: 'M', 
  3049: 'M',
  3157: 'M',
  3162: 'M', 
  3235: 'M',
  3306: 'M',
  3452: 'M', 
  3487: 'M',
  3704: 'M',
  3768: 'M', 
  3855: 'M',
  4036: 'F',
  4338: 'M', 
  4403: 'M',
  4467: 'M',
  4839: 'M', 
  5499: 'M',
  5602: 'M',
  6108: 'M', 
  6131: 'M',
  }
df['Sex'].update(pd.Series(nuevos_valores_area))

In [188]:
#ages to be deduced teens will be 17 years old. When there are more than one number it will be the mean
""" '18 months', '53', '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
'12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
'8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75',
'33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
'20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
'18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35',
'23 & 26', '1', '(adult)', '33 & 37', '25 or 28',
'37, 67, 35, 27,  ? & 27', '21, 34,24 & 35', '30 & 32', '50 & 30',
'17 & 35', 'X', '"middle-age"', '13 or 18', '34 & 19', '33 & 26',
'2 to 3 months', 'MAKE LINE GREEN', ' 43', '81', '"young"',
'7 or 8', '78', '17 & 16', 'F', 'Both 11', '9 or 10', 'young',
'36 & 23', '  ', 'A.M.', '?    &   14', '10 or 12', '31 or 33',
'2½', '13 or 14' """

' \'18 months\', \'53\', \'30s\', \'50s\', \'teen\', \'77\', \'74\', \'28 & 26\', \'5\', \'86\', \'18 or 20\',\n\'12 or 13\', \'46 & 34\', \'28, 23 & 30\', \'Teens\', \'36 & 26\',\n\'8 or 10\', \'84\', \'\xa0 \', \' \', \'30 or 36\', \'6½\', \'21 & ?\', \'75\',\n\'33 or 37\', \'mid-30s\', \'23 & 20\', \' 30\', \'7      &    31\', \' 28\',\n\'20?\', "60\'s", \'32 & 30\', \'16 to 18\', \'87\', \'67\', \'Elderly\',\n\'mid-20s\', \'Ca. 33\', \'74 \', \'45 \', \'21 or 26\', \'20 \', \'>50\',\n\'18 to 22\', \'adult\', \'9 & 12\', \'? & 19\', \'9 months\', \'25 to 35\',\n\'23 & 26\', \'1\', \'(adult)\', \'33 & 37\', \'25 or 28\',\n\'37, 67, 35, 27,  ? & 27\', \'21, 34,24 & 35\', \'30 & 32\', \'50 & 30\',\n\'17 & 35\', \'X\', \'"middle-age"\', \'13 or 18\', \'34 & 19\', \'33 & 26\',\n\'2 to 3 months\', \'MAKE LINE GREEN\', \' 43\', \'81\', \'"young"\',\n\'7 or 8\', \'78\', \'17 & 16\', \'F\', \'Both 11\', \'9 or 10\', \'young\',\n\'36 & 23\', \'  \', \'A.M.\', \'?    &   14\', \'10 or 12\', \'

In [189]:
df.Age.replace({ '18 months': '1', 
  '30s': '35',
  '50s': '50',
  'teen ': '16',
  '18 or 20': '19', 
  '12 or 13': '12',
  '28 23  30': '28',
  'Teens ': '16',
  '36  26': '31', 
  '8 or 10': '9',
  '\xa0 ': 'nan',
  '  ': 'nan',
  '30 or 36': '33', 
  '6½': '6',
  '21  ': '21',
  '33 or 37': '35',
  'mid30s': '35',
  '23  20': '21',
  ' 30': '30', 
  '7          31': 'nan',
  ' 28': '28',
  '32  30': '31',
  '16 to 18': '17', 
  'Elderly': '65',
  'mid20s': '25',
  'Ca 33': '33',
  '74 ': '74',
  '45 ': '45', 
  '21 or 26': '23',
  '20 ': '20',
  '18 to 22': '20',
  'adult': '42',
  '9  12': '10',
  '   19': '19', 
  '9 months': '0',
  ' 25 to 35': '30',
  '23  26': '24',
  '33  37': '35',
  '25 or 28': '26',
  '37 67 35 27    27': '38',
  '21 3424  35': '28',
  '30  32': '31',
  '50  30': '40',
  '17  35': '26',
  'X': 'nan',
  'middleage': '50',
  '13 or 18': '15', 
  '34  19': '26',
  '33  26': '29',
  '2 to 3 months': '0',
  'MAKE LINE GREEN': 'nan', 
  ' 43': '43',
  'young': '3',
  '7 or 8': '7',
  '17  16': '16',
  'F': 'nan',
  'Both 11': '11', 
  '9 or 10': '9',
  '36  23': '29',
  '  ': 'nan',
  'AM': 'nan', 
  '       14': '14',
  '10 or 12': '11',    
  '31 or 33': '32',
  '2½': '2',
  '13 or 14': '13',   
  '40s': '45',
  'Teen': 'nan',
  '60s': '60',
  'teen': 'nan',
  'Teens': 'nan',
  ' ': 'nan',
  '  19': '19',
  '25 to 35': '30'
}, inplace=True)

In [190]:
# Remove non-numeric characters from the 'Age' column and convert to numeric, setting errors='coerce'
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

# Finally, convert the column to integers
df['Age'] = df['Age'].astype('Int64')

In [191]:
# too many unique values
df.Injury.fillna('unknown', inplace=True)

In [192]:
df['Fatal (Y/N)'].fillna('unknown', inplace=True)

df['Fatal (Y/N)'].replace({'M': 'N',
  'UNKNOWN': 'unknown', 
  '2017': 'unknown',
  ' N': 'N',
  'N ': 'N',
  'y': 'Y'
}, inplace=True)

In [193]:

""" 
def is_valid_date(date_str):
    if isinstance(date_str, str):
        try:
            parser.parse(date_str)
            return True
        except ValueError:
            return False
    return False

# Use the function to replace invalid dates with 'unknown'
df['Date'] = df['Date'].apply(lambda x: x if is_valid_date(x) else 'unknown') """


" \ndef is_valid_date(date_str):\n    if isinstance(date_str, str):\n        try:\n            parser.parse(date_str)\n            return True\n        except ValueError:\n            return False\n    return False\n\n# Use the function to replace invalid dates with 'unknown'\ndf['Date'] = df['Date'].apply(lambda x: x if is_valid_date(x) else 'unknown') "

In [194]:
df.sample()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
2146,2147,1998.12.22,1998,Unprovoked,Australia,South australia,Middleton beach,Standing,Megan O'Leary,F,21,2 puncture wounds in left leg,N,15h30,unknown,"The Advertiser, 12/23/1998, p.3; Daily Telegra...",1998.12.22-O'Leary.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1998.12.22,1998.12.22,4157.0,NaN,NaN


In [195]:
columns_to_drop = ['href formula', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']
df = df.drop(columns=columns_to_drop)

In [196]:
df.sample()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href
2606,2607,1991.02.24,1991,Unprovoked,Usa,Oregon,"Neskowin, tillamook county",Surfing,Tony Franciscone,M,38,Calf lacerated & board bitten,N,09h30,5.5 m [18'] white shark,"R. Collier, pp.122-123",1991.02.24-Franciscone_Collier.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [197]:
df.sample()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href
638,639,2013.08.17,2013,Unprovoked,Usa,California,"Pillar point, half-moon bay, san mateo county",Surfing,Wendi Zuccaro,F,<NA>,"No injury, shark bumped surfboard",N,12h40,White shark,R. Collier,2013.08.17-Zaccaro.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [198]:
activity_patterns = {
  'Drinking': r'\bdrink|\bdrunk|\balcohol|\bparty\b',
  'Surfing': r'\bsurf|\bboard|\bpaddle|\bpaddling\b',
  'Diving': r'\bdive|diving|snorkel\b',
  'Swimming': r'\bswim|beach|bath\b',
  'Fishing': r'\bfish\b',
  'Boating': r'\bboat|\boverboard|\bship|\bsail|\bferry|\bwreck',
  'Kayaking': r'\bkayak\b',
}

def categorize_activity(activity):
  if pd.isna(activity):
    return 'unknown'
  if 'overboard' in activity or 'Overboard' in activity:
    return 'Boating'
  for category, pattern in activity_patterns.items():
    if re.search(pattern, activity, re.IGNORECASE):
      return category
  return 'unknown'

df['Activity'] = df['Activity'].apply(categorize_activity)
df['Activity'].unique()

array(['Surfing', 'unknown', 'Diving', 'Swimming', 'Kayaking', 'Fishing',
       'Boating'], dtype=object)

In [199]:
import pandas as pd
import re

def clean_and_standardize_time(time_str):
    if pd.isna(time_str) or time_str == 'unknown':
        return 'unknown'

    time_str = str(time_str).strip().lower()
    
    # Handle known specific patterns
    if re.match(r'\d{2}h\d{2}', time_str):
        return re.sub(r'(\d{2})h(\d{2})', r'\1:\2', time_str)[:5]  # Return the first 5 characters
    
    # Handle 'Morning' and 'Night'
    if 'morning' in time_str:
        return '08:00'[:5]  # Return the first 5 characters
    if 'night' in time_str:
        return '20:00'[:5]  # Return the first 5 characters
    
    # Handle 'Shortly before' entries
    if 'shortly before' in time_str or 'just before' in time_str:
        # Extract the hour from the entry, e.g., '12h00'
        match = re.search(r'(\d{2})h(\d{2})', time_str)
        if match:
            return f"{match.group(1)}:{match.group(2)}"[:5]  # Return the first 5 characters
    
    # Handle ranges
    if re.match(r'\d{2}h\d{2} - \d{2}h\d{2}', time_str):
        # You can calculate the midpoint or handle it as a range
        # For midpoint:
        start, end = time_str.split(' - ')
        start = re.sub(r'(\d{2})h(\d{2})', r'\1:\2', start)[:5]  # Return the first 5 characters
        end = re.sub(r'(\d{2})h(\d{2})', r'\1:\2', end)[:5]  # Return the first 5 characters
        return start
    
    return 'unknown'

# Apply the function to your DataFrame
df['Time'] = df['Time'].apply(clean_and_standardize_time)


In [200]:
df['Time'].unique()

array(['18:00', '14:00', '07:45', 'unknown', '17:00', '08:00', '15:00',
       '08:15', '11:00', '10:30', '10:40', '16:50', '07:00', '09:30',
       '21:50', '09:40', '17:35', '15:30', '07:30', '19:00', '20:00',
       '16:00', '15:01', '12:00', '13:45', '23:30', '09:00', '14:30',
       '18:30', '12:30', '16:30', '18:45', '06:00', '10:00', '10:44',
       '13:19', '13:30', '10:45', '11:20', '11:45', '19:30', '08:30',
       '15:45', '17:34', '17:10', '11:15', '08:50', '17:45', '13:00',
       '10:20', '13:20', '02:00', '09:50', '11:30', '17:30', '10:43',
       '15:15', '15:40', '19:05', '22:00', '16:20', '14:34', '15:25',
       '14:55', '17:46', '15:49', '10:15', '18:15', '04:00', '14:50',
       '13:50', '19:20', '10:25', '16:45', '15:52', '06:15', '14:40',
       '19:45', '10:10', '08:45', '12:10', '18:05', '11:41', '12:25',
       '17:51', '16:12', '12:45', '10:50', '14:15', '09:45', '06:45',
       '05:00', '03:30', '06:30', '17:20', '16:40', '16:18', '11:10',
       '17:50', '1

In [201]:
# just in case reset the index and the case number
df = df.reset_index(drop=True)

df['Case Number'] = [int(e) for e in range(1, len(df)+1)]

In [202]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6299 entries, 0 to 6298
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             6299 non-null   int64 
 1   Date                    6296 non-null   object
 2   Year                    6170 non-null   Int64 
 3   Type                    6299 non-null   object
 4   Country                 6299 non-null   object
 5   Area                    6299 non-null   object
 6   Location                6299 non-null   object
 7   Activity                6299 non-null   object
 8   Name                    6299 non-null   object
 9   Sex                     6299 non-null   object
 10  Age                     3410 non-null   Int64 
 11  Injury                  6299 non-null   object
 12  Fatal (Y/N)             6299 non-null   object
 13  Time                    6299 non-null   object
 14  Species                 6299 non-null   object
 15  Inve

In [203]:
df.to_csv('../data/shark_attacks_clean.csv', index=False)